In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, median_absolute_error
import matplotlib.pyplot as plt

# Загрузка данных
taxi_data = pd.read_csv("data/train.csv")

# Преобразование даты и времени
taxi_data['pickup_datetime'] = pd.to_datetime(taxi_data['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')

# Добавление новых признаков
taxi_data['pickup_date'] = taxi_data['pickup_datetime'].dt.date
taxi_data['pickup_hour'] = taxi_data['pickup_datetime'].dt.hour
taxi_data['pickup_day_of_week'] = taxi_data['pickup_datetime'].dt.day_of_week

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Вычисляет расстояние между двумя точками на поверхности Земли с использованием формулы гаверсина.
    """
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    return 2 * 6371 * np.arcsin(np.sqrt(a))

# Расчет расстояния между координатами
taxi_data['haversine_distance'] = haversine_distance(
    taxi_data['pickup_latitude'], taxi_data['pickup_longitude'],
    taxi_data['dropoff_latitude'], taxi_data['dropoff_longitude']
)

def calculate_direction(lat1, lon1, lat2, lon2):
    """
    Вычисляет направление движения между двумя координатами в градусах.
    """
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    x = np.sin(dlon) * np.cos(lat2)
    y = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(dlon)
    return np.degrees(np.arctan2(x, y))

# Расчет направления между координатами
taxi_data['direction'] = calculate_direction(
    taxi_data['pickup_latitude'], taxi_data['pickup_longitude'],
    taxi_data['dropoff_latitude'], taxi_data['dropoff_longitude']
)

# Фильтрация и преобразование данных для модели
features = ['vendor_id', 'passenger_count', 'pickup_hour', 'pickup_day_of_week',
            'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude',
            'haversine_distance', 'direction']
X = taxi_data[features]
y = np.log1p(taxi_data['trip_duration'])

# Разделение данных на тренировочную и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормализация факторов
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Построение модели линейной регрессии
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled, y_train)

# Построение модели деревьев решений
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X_train, y_train)

# Построение модели полиномиальной регрессии 2-ой степени
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_val_poly = poly.transform(X_val)

scaler_poly = StandardScaler()
X_train_poly_scaled = scaler_poly.fit_transform(X_train_poly)
X_val_poly_scaled = scaler_poly.transform(X_val_poly)

poly_reg = LinearRegression()
poly_reg.fit(X_train_poly_scaled, y_train)

# Построение модели Ridge регрессии с полиномиальными признаками
ridge_reg = Ridge(alpha=1.0)
ridge_reg.fit(X_train_poly_scaled, y_train)

# Построение модели случайного леса
rf_reg = RandomForestRegressor(
    n_estimators=200,
    max_depth=12,
    criterion='squared_error',
    min_samples_split=20,
    random_state=42
)
rf_reg.fit(X_train_scaled, y_train)

# Построение модели градиентного бустинга
gbr_reg = GradientBoostingRegressor(
    learning_rate=0.5,
    n_estimators=100,
    max_depth=6,
    min_samples_split=30,
    random_state=42
)
gbr_reg.fit(X_train_scaled, y_train)

def rmsle(y_true, y_pred):
    """
    Вычисляет Root Mean Squared Log Error (RMSLE) между истинными и предсказанными значениями.
    """
    return np.sqrt(mean_squared_error(y_true, y_pred, squared=False))

# Предсказания и оценка метрики RMSLE для линейной регрессии
y_train_pred_lin = lin_reg.predict(X_train_scaled)
y_val_pred_lin = lin_reg.predict(X_val_scaled)
rmsle_train_lin = rmsle(y_train, y_train_pred_lin)
rmsle_val_lin = rmsle(y_val, y_val_pred_lin)

# Предсказания и оценка метрики RMSLE для деревьев решений
y_train_pred_tree = tree_reg.predict(X_train)
y_val_pred_tree = tree_reg.predict(X_val)
rmsle_train_tree = rmsle(y_train, y_train_pred_tree)
rmsle_val_tree = rmsle(y_val, y_val_pred_tree)

# Предсказания и оценка метрики RMSLE для полиномиальной регрессии
y_train_pred_poly = poly_reg.predict(X_train_poly_scaled)
y_val_pred_poly = poly_reg.predict(X_val_poly_scaled)
rmsle_train_poly = rmsle(y_train, y_train_pred_poly)
rmsle_val_poly = rmsle(y_val, y_val_pred_poly)

# Предсказания и оценка метрики RMSLE для Ridge регрессии
y_train_pred_ridge = ridge_reg.predict(X_train_poly_scaled)
y_val_pred_ridge = ridge_reg.predict(X_val_poly_scaled)
rmsle_train_ridge = rmsle(y_train, y_train_pred_ridge)
rmsle_val_ridge = rmsle(y_val, y_val_pred_ridge)

# Предсказания и оценка метрики RMSLE для случайного леса
y_train_pred_rf = rf_reg.predict(X_train_scaled)
y_val_pred_rf = rf_reg.predict(X_val_scaled)
rmsle_train_rf = rmsle(y_train, y_train_pred_rf)
rmsle_val_rf = rmsle(y_val, y_val_pred_rf)

# Предсказания и оценка метрики RMSLE для градиентного бустинга
y_train_pred_gbr = gbr_reg.predict(X_train_scaled)
y_val_pred_gbr = gbr_reg.predict(X_val_scaled)
rmsle_train_gbr = rmsle(y_train, y_train_pred_gbr)
rmsle_val_gbr = rmsle(y_val, y_val_pred_gbr)

print(f'Линейная регрессия - RMSLE на тренировочной выборке: {rmsle_train_lin:.2f}')
print(f'Линейная регрессия - RMSLE на валидационной выборке: {rmsle_val_lin:.2f}')
print(f'Полиномиальная регрессия - RMSLE на тренировочной выборке: {rmsle_train_poly:.2f}')
print(f'Полиномиальная регрессия - RMSLE на валидационной выборке: {rmsle_val_poly:.2f}')
print(f'Дерево решений - RMSLE на тренировочной выборке: {rmsle_train_tree:.2f}')
print(f'Дерево решений - RMSLE на валидационной выборке: {rmsle_val_tree:.2f}')
print(f'Ridge регрессия - RMSLE на тренировочной выборке: {rmsle_train_ridge:.2f}')
print(f'Ridge регрессия - RMSLE на валидационной выборке: {rmsle_val_ridge:.2f}')
print(f'Случайный лес - RMSLE на тренировочной выборке: {rmsle_train_rf:.2f}')
print(f'Случайный лес - RMSLE на валидационной выборке: {rmsle_val_rf:.2f}')
print(f'Градиентный бустинг - RMSLE на тренировочной выборке: {rmsle_train_gbr:.2f}')
print(f'Градиентный бустинг - RMSLE на валидационной выборке: {rmsle_val_gbr:.2f}')

# Нахождение модели с наименьшим RMSLE на валидационной выборке
results = {
    'Линейная регрессия': {'RMSLE на валидационной выборке': rmsle_val_lin},
    'Полиномиальная регрессия': {'RMSLE на валидационной выборке': rmsle_val_poly},
    'Дерево решений': {'RMSLE на валидационной выборке': rmsle_val_tree},
    'Ridge регрессия': {'RMSLE на валидационной выборке': rmsle_val_ridge},
    'Случайный лес': {'RMSLE на валидационной выборке': rmsle_val_rf},
    'Градиентный бустинг': {'RMSLE на валидационной выборке': rmsle_val_gbr}
}

best_model_name = min(results, key=lambda x: results[x]['RMSLE на валидационной выборке'])
print(f'Лучшая модель на валидационной выборке: {best_model_name}')
